In [1]:
from selenium import webdriver
import pandas as pd
import numpy as np
from selenium.webdriver.common.by import By
import time 
from selenium.webdriver import Chrome 
import requests
from bs4 import BeautifulSoup as bs
from tqdm import tqdm_notebook

In [2]:
def getPageLinks(page_number):
    """

       Возвращает список ссылок на сумки, полученный с текущей страницы

        page_number: int/string
            номер страницы для парсинга
            

    """
    # составляем ссылку на страницу поиска
    page_link = "https://oskelly.ru/catalog/zhenskoe/sumki?page={}".format(page_number)
    response = requests.get(page_link)
    
    html = response.content
    soup = bs(html,'html.parser')

    # наконец, ищем ссылки на мемы и очищаем их от ненужных тэгов
    bag_links = soup.find_all('a', attrs = {'class':'osk-product-card__card-img osk-product-card__card-img_small'})
    bag_links = ['https://oskelly.ru' + link.attrs['href'] for link in bag_links]

    return bag_links[:]


In [3]:
#получаем ссылки на все сумки, которые будут у нас в базе
bag_links=[]

In [4]:
for i in range (1, 51):
    bag_links+=getPageLinks(i)

In [42]:
print(bag_links[535])

https://oskelly.ru/products/sumka-s-korotkimi-ruchkami-hermespre-owned-pre-owned-2033113


In [38]:
driver = webdriver.Chrome()
arr_dic=[]
#переходим на страницу сумки
url = bag_links[0]
driver.get(url)
time.sleep(5) 
        
#получаем цену
pr = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product__price')]
price = int(''.join([c for c in pr[0] if c.isdigit()]))

#нажиаем на +, чтобы открыть детали сумки
button = driver.find_elements(By.CLASS_NAME, 'osk-collapse-item__plus')
cl = button[0].click()
time.sleep(3)
det = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product-attributes')]
string = det[0]
        
arr = string.split('\n')
d=dict()

for i in range (0,len(arr)-1,2):
    d[arr[i]]=arr[i+1]
d['Цена'] = price
arr_dic.append(d)
driver.close()

print(arr_dic)

[{'Размер': 'INT L', 'Раздел': 'Женское', 'Категория': 'Сумки с короткими ручками', 'Бренд': 'MARC JACOBS', 'Материал сумок': 'Экзотическая кожа', 'Цвет': 'Серебряный', 'Длина ручки': 'Средние ручки', 'Наличие пыльника': 'Да', 'Состояние товара': 'Отличное состояние', 'Продавец': 'Частный продавец', 'Oskelly ID': '1993293', 'Цена': 52000}]


In [46]:

#data = pd.DataFrame(columns=['Размер', 'Раздел', 'Категория', 'Бренд', 'Материал сумок', 'Цвет', 'Длина ручки', 'Состояние товара', 'Продавец', 'Oskelly ID'])
arr_dic=[]
a = list(range(1200))
j = 0
prices=[]
for i in tqdm_notebook(a):
    driver = webdriver.Chrome()
    #переходим на страницу сумки
    url = bag_links[j]
    driver.get(url)
    time.sleep(5) 
        
    #получаем цену
    pr = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product__price')]
    price = int(''.join([c for c in pr[0] if c.isdigit()]))
    
    #нажиаем на +, чтобы открыть детали сумки
    button = driver.find_elements(By.CLASS_NAME, 'osk-collapse-item__plus')
    cl = button[0].click()
    time.sleep(3)
    det = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product-attributes')]
    string = det[0]
        
    arr = string.split('\n')
    d=dict()
    for k in range (0,len(arr)-1,2):
        d[arr[k]]=arr[k+1]
    d['Цена']=price
    arr_dic.append(d)
    driver.close()
    j+=1

C:\Users\Алина\AppData\Local\Temp\ipykernel_28960\2476087600.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(a):


  0%|          | 0/1200 [00:00<?, ?it/s]

IndexError: list index out of range

In [48]:
df1 = pd.DataFrame(arr_dic)
df1

,Размер,Раздел,Категория,Бренд,Материал сумок,Цвет,Длина ручки,Наличие пыльника,Состояние товара,Продавец,Oskelly ID,Цена,Модель,Винтаж,Наличие коробки,Наличие сертификата
0,INT L,Женское,Сумки с короткими ручками,MARC JACOBS,Экзотическая кожа,Серебряный,Средние ручки,Да,Отличное состояние,Частный продавец,1993293,52000,NaN,NaN,NaN,NaN
1,INT S,Женское,Клатчи и вечерние сумки,VERSACE,Кожа,Черный,Короткие ручки,NaN,Отличное состояние,Частный продавец,1993444,38000,NaN,NaN,NaN,NaN
2,INT M,Женское,Сумки через плечо,BALENCIAGA,Кожа,Желтый,Длинный ремень,NaN,Новое с биркой,Частный продавец,2035063,345000,Crush,NaN,NaN,NaN
3,INT U,Женское,Сумки с короткими ручками,BURBERRY,Кожа,Коричневый,Средние ручки,NaN,Отличное состояние,Частный продавец,1941649,27000,NaN,Да,NaN,NaN
4,INT M,Женское,Сумки с короткими ручками,KARL LAGERFELD,Искусственная кожа,Мульти,Средние ручки,NaN,Новое с биркой,Частный продавец,2008104,17950,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
568,INT L,Женское,Сумки через плечо,BVLGARI,Кожа,Черный,Длинный ремень,NaN,Хорошее состояние,Частный продавец,1735035,158000,Serpenti,NaN,Да,NaN
569,INT M,Женское,Сумки через плечо,BVLGARI,Кожа,Зеленый,Длинный ремень,Да,Отличное состояние,Частный продавец,1822079,148000,Serpenti,NaN,Да,NaN
570,INT S,Женское,Клатчи и вечерние сумки,GIVENCHY,Кожа,Розовый,Короткие ручки,NaN,Отличное состояние,Частный продавец,1966065,13000,NaN,NaN,NaN,NaN
571,INT XS,Женское,Сумки с короткими ручками,CELINE PRE-OWNED,Ткань,Белый,Короткие ручки,NaN,Отличное состояние,Частный продавец,1993088,30000,NaN,Да,NaN,NaN


In [51]:
arr_dic=[]
a = list(range(626))
j = 575
prices=[]
for i in tqdm_notebook(a):
    driver = webdriver.Chrome()
    #переходим на страницу сумки
    url = bag_links[j]
    driver.get(url)
    time.sleep(5) 
        
    #получаем цену
    pr = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product__price')]
    price = int(''.join([c for c in pr[0] if c.isdigit()]))
    
    #нажиаем на +, чтобы открыть детали сумки
    button = driver.find_elements(By.CLASS_NAME, 'osk-collapse-item__plus')
    cl = button[0].click()
    time.sleep(3)
    det = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product-attributes')]
    string = det[0]
        
    arr = string.split('\n')
    d=dict()
    for k in range (0,len(arr)-1,2):
        d[arr[k]]=arr[k+1]
    d['Цена']=price
    arr_dic.append(d)
    driver.close()
    j+=1

C:\Users\Алина\AppData\Local\Temp\ipykernel_28960\948918670.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(a):


  0%|          | 0/626 [00:00<?, ?it/s]

IndexError: list index out of range

In [52]:
df2 = pd.DataFrame(arr_dic)
df2

,Размер,Раздел,Категория,Бренд,Модель,Материал сумок,Цвет,Длина ручки,Наличие пыльника,Состояние товара,Продавец,Oskelly ID,Цена,Винтаж,Наличие сертификата,Наличие коробки
0,INT S,Женское,Сумки с короткими ручками,BALENCIAGA,Hourglass,Кожа,Розовый,Короткие ручки,Да,Отличное состояние,Частный продавец,1449159,120000,NaN,NaN,NaN
1,INT M,Женское,Сумки тоут,PRADA,Tessuto,Ткань,Темно-синий,Средние ручки,NaN,Отличное состояние,Частный продавец,1684684,65000,NaN,NaN,NaN
2,INT M,Женское,Сумки тоут,OFF-WHITE,NaN,Другое,Черный,Средние ручки,NaN,Хорошее состояние,Частный продавец,2030586,17500,NaN,NaN,NaN
3,INT M,Женское,Сумки с короткими ручками,LOUIS VUITTON PRE-OWNED,Speedy,Другое,Бежевый,Короткие ручки,NaN,Отличное состояние,Частный продавец,2004404,56000,NaN,NaN,NaN
4,INT M,Женское,Сумки с короткими ручками,CELINE PRE-OWNED,Belt,Кожа,Черный,Длинный ремень,NaN,Отличное состояние,Частный продавец,2033307,180000,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
130,INT U,Женское,Сумки тоут,EMPORIO ARMANI,NaN,Другое,Черный,Средние ручки,NaN,Новое с биркой,Бутик,2023153,67900,NaN,NaN,NaN
131,INT S,Женское,Клатчи и вечерние сумки,BOTTEGA VENETA,NaN,Кожа,Черный,Длинный ремень,NaN,Новое с биркой,Частный продавец,2031211,100000,NaN,NaN,NaN
132,INT S,Женское,Сумки с короткими ручками,HERMES PRE-OWNED,Herbag,Другое,Оранжевый,Длинный ремень,Да,Новое с биркой,Частный продавец,2032493,470000,NaN,NaN,Да
133,INT U,Женское,Сумки тоут,EMPORIO ARMANI,NaN,Ткань,Бежевый,Средние ручки,NaN,Новое с биркой,Бутик,2023154,55900,NaN,NaN,NaN


In [53]:
arr_dic=[]
a = list(range(491))
j = 711
prices=[]
for i in tqdm_notebook(a):
    driver = webdriver.Chrome()
    #переходим на страницу сумки
    url = bag_links[j]
    driver.get(url)
    time.sleep(5) 
        
    #получаем цену
    pr = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product__price')]
    price = int(''.join([c for c in pr[0] if c.isdigit()]))
    
    #нажиаем на +, чтобы открыть детали сумки
    button = driver.find_elements(By.CLASS_NAME, 'osk-collapse-item__plus')
    cl = button[0].click()
    time.sleep(3)
    det = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product-attributes')]
    string = det[0]
        
    arr = string.split('\n')
    d=dict()
    for k in range (0,len(arr)-1,2):
        d[arr[k]]=arr[k+1]
    d['Цена']=price
    arr_dic.append(d)
    driver.close()
    j+=1

C:\Users\Алина\AppData\Local\Temp\ipykernel_28960\3391479603.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(a):


  0%|          | 0/491 [00:00<?, ?it/s]

IndexError: list index out of range

In [54]:
df3 = pd.DataFrame(arr_dic)
df3

,Размер,Раздел,Категория,Бренд,Материал сумок,Цвет,Длина ручки,Состояние товара,Продавец,Oskelly ID,Цена,Модель,Наличие коробки,Наличие пыльника,Наличие сертификата,Винтаж
0,INT S,Женское,Сумки с короткими ручками,LORO PIANA,Кожа,Коричневый,Короткие ручки,Новое с биркой,Частный продавец,2031368,390000,NaN,NaN,NaN,NaN,NaN
1,INT U,Женское,Сумки через плечо,EMPORIO ARMANI,Ткань,Бежевый,Средние ручки,Новое с биркой,Бутик,2023156,45900,NaN,NaN,NaN,NaN,NaN
2,INT XS,Женское,Сумки тоут,HERMES PRE-OWNED,Кожа,Бежевый,Короткие ручки,Новое с биркой,Частный продавец,1985836,350000,Picotin,NaN,NaN,NaN,NaN
3,INT S,Женское,Сумки с короткими ручками,HERMES PRE-OWNED,Шелк,Темно-синий,Короткие ручки,Новое с биркой,Частный продавец,2032352,334000,NaN,Да,Да,NaN,NaN
4,INT U,Женское,Сумки через плечо,VERSACE JEANS COUTURE,Искусственная кожа,Черный,Длинный ремень,Новое с биркой,Бутик,1318141,20930,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,INT U,Женское,Сумки с короткими ручками,HERMES PRE-OWNED,Кожа,Оранжевый,Средние ручки,Хорошее состояние,Частный продавец,2031680,725000,Birkin,NaN,NaN,NaN,NaN
247,INT U,Женское,Сумки через плечо,PRADA,Кожа,Белый,Средние ручки,Новое с биркой,Частный продавец,2020827,307619,NaN,NaN,NaN,NaN,NaN
248,INT U,Женское,Сумки через плечо,PRADA,Кожа,Черный,Средние ручки,Новое с биркой,Частный продавец,2020823,307619,NaN,NaN,NaN,NaN,NaN
249,INT L,Женское,Сумки тоут,LOVE MOSCHINO,Искусственная кожа,Черный,Средние ручки,Новое с биркой,Частный продавец,1906449,22450,NaN,NaN,NaN,NaN,NaN


In [58]:
arr_dic=[]
a = list(range(85))
j = 968
prices=[]
for i in tqdm_notebook(a):
    driver = webdriver.Chrome()
    #переходим на страницу сумки
    url = bag_links[j]
    driver.get(url)
    time.sleep(5) 
        
    #получаем цену
    pr = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product__price')]
    price = int(''.join([c for c in pr[0] if c.isdigit()]))
    
    #нажиаем на +, чтобы открыть детали сумки
    button = driver.find_elements(By.CLASS_NAME, 'osk-collapse-item__plus')
    cl = button[0].click()
    time.sleep(3)
    det = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product-attributes')]
    string = det[0]
        
    arr = string.split('\n')
    d=dict()
    for k in range (0,len(arr)-1,2):
        d[arr[k]]=arr[k+1]
    d['Цена']=price
    arr_dic.append(d)
    driver.close()
    j+=1

C:\Users\Алина\AppData\Local\Temp\ipykernel_28960\3532264920.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(a):


  0%|          | 0/85 [00:00<?, ?it/s]

IndexError: list index out of range

In [59]:
df4 = pd.DataFrame(arr_dic)
df4

,Размер,Раздел,Категория,Бренд,Материал сумок,Цвет,Длина ручки,Состояние товара,Продавец,Oskelly ID,Цена,Наличие пыльника,Модель,Винтаж,Наличие коробки,Наличие сертификата
0,INT U,Женское,Сумки через плечо,MIU MIU,Кожа,Черный,Длинный ремень,Новое с биркой,Частный продавец,2020784,153810,NaN,NaN,NaN,NaN,NaN
1,INT U,Женское,Сумки тоут,SAINT LAURENT,Кожа,Черный,Длинный ремень,Новое с биркой,Частный продавец,2020769,230714,NaN,NaN,NaN,NaN,NaN
2,INT S,Женское,Сумки через плечо,MIU MIU,Кожа,Серый,Длинный ремень,Новое с биркой,Частный продавец,2030778,209000,NaN,NaN,NaN,NaN,NaN
3,INT M,Женское,Сумки тоут,DKNY,Кожа,Черный,Длинный ремень,Отличное состояние,Частный продавец,2029370,13000,Да,NaN,NaN,NaN,NaN
4,INT U,Женское,Сумки через плечо,SAINT LAURENT,Другое,Бежевый,Длинный ремень,Новое с биркой,Частный продавец,2020763,148750,NaN,NaN,NaN,NaN,NaN
5,INT M,Женское,Сумки с короткими ручками,BALENCIAGA,Кожа,Коричневый,Длинный ремень,Отличное состояние,Частный продавец,2029281,155000,NaN,Neo Classic City,NaN,NaN,NaN
6,INT S,Женское,Сумки с короткими ручками,PRADA,Другое,Черный,Короткие ручки,Отличное состояние,Частный продавец,2025836,36000,NaN,NaN,NaN,NaN,NaN
7,INT U,Женское,Сумки через плечо,SAINT LAURENT,Кожа,Черный,Средние ручки,Новое с биркой,Частный продавец,2020750,235714,NaN,Le 5 a 7,NaN,NaN,NaN
8,INT S,Женское,Сумки через плечо,MIU MIU,Кожа,Черный,Длинный ремень,Новое с биркой,Частный продавец,2030794,152000,NaN,NaN,NaN,NaN,NaN
9,INT XS,Женское,Сумки тоут,MIU MIU,Ткань,Другое,Длинный ремень,Новое с биркой,Частный продавец,2030819,259000,NaN,NaN,NaN,NaN,NaN


In [60]:
arr_dic=[]
a = list(range(62))
j = 998
prices=[]
for i in tqdm_notebook(a):
    driver = webdriver.Chrome()
    #переходим на страницу сумки
    url = bag_links[j]
    driver.get(url)
    time.sleep(5) 
        
    #получаем цену
    pr = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product__price')]
    price = int(''.join([c for c in pr[0] if c.isdigit()]))
    
    #нажиаем на +, чтобы открыть детали сумки
    button = driver.find_elements(By.CLASS_NAME, 'osk-collapse-item__plus')
    cl = button[0].click()
    time.sleep(3)
    det = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product-attributes')]
    string = det[0]
        
    arr = string.split('\n')
    d=dict()
    for k in range (0,len(arr)-1,2):
        d[arr[k]]=arr[k+1]
    d['Цена']=price
    arr_dic.append(d)
    driver.close()
    j+=1

C:\Users\Алина\AppData\Local\Temp\ipykernel_28960\1585963244.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(a):


  0%|          | 0/62 [00:00<?, ?it/s]

IndexError: list index out of range

### Можно заметить, что у меня все крашится иногда. Проблема в том, что сумки покупают, объявления пропадают. В то же время постоянно появляются новые предложения из-за чего старые объявления сдвигаются по страницам, поэтому мной было принято решение все-таки собрать зафиксированное количество ссылок, а в таком случае таких поломок к сожалению не избежать(((

In [61]:
df5 = pd.DataFrame(arr_dic)
df5

,Размер,Раздел,Категория,Бренд,Материал сумок,Цвет,Длина ручки,Состояние товара,Продавец,Oskelly ID,Цена,Наличие коробки,Наличие пыльника,Модель,Наличие сертификата
0,INT M,Женское,Сумки с короткими ручками,BURBERRY,Другое,Другое,Длинный ремень,Новое с биркой,Частный продавец,2017260,170000,NaN,NaN,NaN,NaN
1,INT XS,Женское,Сумки с короткими ручками,DIESEL,Хлопок,Зеленый,Средние ручки,Новое с биркой,"Частный, есть возврат",2016656,60000,Да,Да,NaN,NaN
2,INT S,Женское,Сумки через плечо,TOD'S,Кожа,Белый,Длинный ремень,Отличное состояние,Частный продавец,2029071,55000,NaN,NaN,NaN,NaN
3,INT XS,Женское,Сумки через плечо,BOTTEGA VENETA,Кожа,Черный,Средние ручки,Отличное состояние,Частный продавец,2028698,60000,NaN,NaN,NaN,NaN
4,INT M,Женское,Сумки тоут,JIMMY CHOO,Другое,Бежевый,Длинный ремень,Новое с биркой,Частный продавец,2018252,155300,NaN,NaN,NaN,NaN
5,INT M,Женское,Сумки тоут,COCCINELLE,Кожа,Коричневый,Средние ручки,Отличное состояние,Частный продавец,2028465,15000,NaN,NaN,NaN,NaN
6,INT S,Женское,Сумки через плечо,KARL LAGERFELD,Искусственная кожа,Голубой,Длинный ремень,Новое с биркой,Частный продавец,2028846,21000,NaN,Да,NaN,NaN
7,INT U,Женское,Сумки через плечо,GUCCI,Другое,Коричневый,Длинный ремень,Новое с биркой,Частный продавец,2020536,131250,NaN,NaN,Blondie,NaN
8,INT M,Женское,Сумки через плечо,LOVE MOSCHINO,Другое,Черный,Длинный ремень,Отличное состояние,Частный продавец,2028888,13000,NaN,Да,NaN,NaN
9,INT S,Женское,Сумки через плечо,MARC JACOBS,Лакированная кожа,Фуксия,Длинный ремень,Отличное состояние,Частный продавец,2028978,35000,NaN,NaN,The Snapshot Bag,Да


In [63]:
a = list(range(41))
j = 1021
prices=[]
for i in tqdm_notebook(a):
    driver = webdriver.Chrome()
    #переходим на страницу сумки
    url = bag_links[j]
    driver.get(url)
    time.sleep(3) 
        
    #получаем цену
    pr = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product__price')]
    price = int(''.join([c for c in pr[0] if c.isdigit()]))
    
    #нажиаем на +, чтобы открыть детали сумки
    button = driver.find_elements(By.CLASS_NAME, 'osk-collapse-item__plus')
    cl = button[0].click()
    time.sleep(3)
    det = [x.text for x in driver.find_elements(By.CLASS_NAME, 'osk-product-attributes')]
    string = det[0]
        
    arr = string.split('\n')
    d=dict()
    for k in range (0,len(arr)-1,2):
        d[arr[k]]=arr[k+1]
    d['Цена']=price
    arr_dic.append(d)
    driver.close()
    j+=1

C:\Users\Алина\AppData\Local\Temp\ipykernel_28960\3380121965.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(a):


  0%|          | 0/41 [00:00<?, ?it/s]

IndexError: list index out of range

In [64]:
df6 = pd.DataFrame(arr_dic)
df6

,Размер,Раздел,Категория,Бренд,Материал сумок,Цвет,Длина ручки,Состояние товара,Продавец,Oskelly ID,Цена,Наличие коробки,Наличие пыльника,Модель,Наличие сертификата,Винтаж
0,INT M,Женское,Сумки с короткими ручками,BURBERRY,Другое,Другое,Длинный ремень,Новое с биркой,Частный продавец,2017260,170000,NaN,NaN,NaN,NaN,NaN
1,INT XS,Женское,Сумки с короткими ручками,DIESEL,Хлопок,Зеленый,Средние ручки,Новое с биркой,"Частный, есть возврат",2016656,60000,Да,Да,NaN,NaN,NaN
2,INT S,Женское,Сумки через плечо,TOD'S,Кожа,Белый,Длинный ремень,Отличное состояние,Частный продавец,2029071,55000,NaN,NaN,NaN,NaN,NaN
3,INT XS,Женское,Сумки через плечо,BOTTEGA VENETA,Кожа,Черный,Средние ручки,Отличное состояние,Частный продавец,2028698,60000,NaN,NaN,NaN,NaN,NaN
4,INT M,Женское,Сумки тоут,JIMMY CHOO,Другое,Бежевый,Длинный ремень,Новое с биркой,Частный продавец,2018252,155300,NaN,NaN,NaN,NaN,NaN
5,INT M,Женское,Сумки тоут,COCCINELLE,Кожа,Коричневый,Средние ручки,Отличное состояние,Частный продавец,2028465,15000,NaN,NaN,NaN,NaN,NaN
6,INT S,Женское,Сумки через плечо,KARL LAGERFELD,Искусственная кожа,Голубой,Длинный ремень,Новое с биркой,Частный продавец,2028846,21000,NaN,Да,NaN,NaN,NaN
7,INT U,Женское,Сумки через плечо,GUCCI,Другое,Коричневый,Длинный ремень,Новое с биркой,Частный продавец,2020536,131250,NaN,NaN,Blondie,NaN,NaN
8,INT M,Женское,Сумки через плечо,LOVE MOSCHINO,Другое,Черный,Длинный ремень,Отличное состояние,Частный продавец,2028888,13000,NaN,Да,NaN,NaN,NaN
9,INT S,Женское,Сумки через плечо,MARC JACOBS,Лакированная кожа,Фуксия,Длинный ремень,Отличное состояние,Частный продавец,2028978,35000,NaN,NaN,The Snapshot Bag,Да,NaN


In [65]:
result = pd.concat([df1, df2, df3, df4, df5, df6])

result

,Размер,Раздел,Категория,Бренд,Материал сумок,Цвет,Длина ручки,Наличие пыльника,Состояние товара,Продавец,Oskelly ID,Цена,Модель,Винтаж,Наличие коробки,Наличие сертификата
0,INT L,Женское,Сумки с короткими ручками,MARC JACOBS,Экзотическая кожа,Серебряный,Средние ручки,Да,Отличное состояние,Частный продавец,1993293,52000,NaN,NaN,NaN,NaN
1,INT S,Женское,Клатчи и вечерние сумки,VERSACE,Кожа,Черный,Короткие ручки,NaN,Отличное состояние,Частный продавец,1993444,38000,NaN,NaN,NaN,NaN
2,INT M,Женское,Сумки через плечо,BALENCIAGA,Кожа,Желтый,Длинный ремень,NaN,Новое с биркой,Частный продавец,2035063,345000,Crush,NaN,NaN,NaN
3,INT U,Женское,Сумки с короткими ручками,BURBERRY,Кожа,Коричневый,Средние ручки,NaN,Отличное состояние,Частный продавец,1941649,27000,NaN,Да,NaN,NaN
4,INT M,Женское,Сумки с короткими ручками,KARL LAGERFELD,Искусственная кожа,Мульти,Средние ручки,NaN,Новое с биркой,Частный продавец,2008104,17950,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,INT L,Женское,Сумки с короткими ручками,VALENTINO,Кожа,Черный,Короткие ручки,NaN,Отличное состояние,Частный продавец,2020146,80000,NaN,NaN,NaN,NaN
50,INT XS,Женское,Сумки через плечо,HERMES PRE-OWNED,Кожа,Другое,Длинный ремень,Да,Новое с биркой,Частный продавец,1961154,1100000,Jypsiere,NaN,Да,NaN
51,INT M,Женское,Сумки через плечо,BOTTEGA VENETA,Кожа,Коричневый,Длинный ремень,NaN,Отличное состояние,Частный продавец,2000414,79000,Pouch,NaN,NaN,NaN
52,INT XS,Женское,Сумки через плечо,HERMES PRE-OWNED,Кожа,Зеленый,Длинный ремень,NaN,Новое с биркой,Частный продавец,1448898,360000,Evelyne,NaN,NaN,NaN


In [66]:
result.to_csv(r'bag_data.csv', index = False)